In [1]:
from newspaper import Article
import requests
import pynytimes
from pynytimes import NYTAPI
import pandas as pd
import time

https://github.com/evadrichter/nyt_api
https://developer.nytimes.com/docs/articlesearch-product/1/overview


In [7]:
keywords = ["Israel", "Palestine", "Gaza", "Hamas"]
begin_date = "20231101"
end_date = "20231130"
# keywords = ["Hamas"]
# begin_date = "20231001"
# end_date = "20231031"

api_key = "ITAgd7CvroJ9CGxd7BjDPjcsve5QdD1s"
nytapi = pynytimes.NYTAPI(api_key, parse_dates=True)

texts = {}

for keyword in keywords:
    print("Keyword: ", keyword)
    time.sleep(12)
    # find num hits
    api_url = f"https://api.nytimes.com/svc/search/v2/articlesearch.json?q={keyword}&begin_date={begin_date}&end_date={end_date}&api-key={api_key}"
    response = requests.get(api_url)
    if response.status_code == 200:
        # Parse and use the response data
        data = response.json()
        hits = data["response"]["meta"]["hits"]
        print("hits: ", hits)
    else:
        print(f"Error: {response.status_code}, {response.text}")
        
    # get urls
    for page in range( int(hits) // 10 ):
        print("Page: ", page)
        time.sleep(12)
        api_url = f"https://api.nytimes.com/svc/search/v2/articlesearch.json?q={keyword}&begin_date={begin_date}&end_date={end_date}&api-key={api_key}&page={page}"
        response = requests.get(api_url)
        data = response.json()
        for d in data["response"]["docs"]:
            if response.status_code == 200:
                texts[d["web_url"]] = (d["abstract"], d["lead_paragraph"])
                print(len(texts.keys()), ": ", d["web_url"])
                print("     ", d["abstract"])
                print("     ", d["lead_paragraph"])
            else:
                print(f"Error: {response.status_code}, {response.text}")

urls = list(texts.keys())
abstract = []
lead_paragraph = []
for url in urls:
    abstract.append(texts[url][0])
    lead_paragraph.append(texts[url][1])
    
df_data = {"url" : urls, 
           "abstract": abstract,
           "lead_paragraph": lead_paragraph}
df = pd.DataFrame(df_data)
    
# exporting the data to csv:
df.to_csv('../data/NYT_data_Nov.csv')
# df.to_csv(f'NYT_data_{keyword}.csv')

Keyword:  Israel
hits:  832
Page:  0
1 :  https://www.nytimes.com/2023/11/25/world/middleeast/israel-gaza-death-toll.html
      Even a conservative assessment of the reported Gaza casualty figures shows that the rate of death during Israel’s assault has few precedents in this century, experts say.
      Israel has cast the deaths of civilians in the Gaza Strip as a regrettable but unavoidable part of modern conflict, pointing to the heavy human toll from military campaigns the United States itself once waged in Iraq and Syria.
2 :  https://www.nytimes.com/2023/11/13/world/middleeast/israel-evacuees-border-attacks.html
      More than 125,000 Israeli evacuees have been resettled from their communities along the dangerous borders with Lebanon and Gaza in the largest internal displacement in the country’s history.
      A joyful clamor echoed in the ballroom of the Golden Crown Hotel. Kindergarten was in full swing for 30 children from Arab al-Aramshe, a village next to Israel’s border wi

In [6]:
link_csv = '../data/NYT_data_Nov.csv'
final_csv = '../data/NYT_text_data_Nov.csv'

df = pd.read_csv(link_csv, index_col=0)

full_texts = []
start = 823

for i in range(start, len(list(df.url))):
    url = df.url[i]
    print(i, ": ", url)
    article = Article(url)
    time.sleep(6)
    article.download()
    article.parse()
    text = article.text
    full_texts.append(text)
    df_row = {"url" : [url], 
                "full_text": [text]}
    if i == 0:
        df_curr = pd.DataFrame(df_row)
        df_curr.to_csv(final_csv)  
    else:
        df_curr = pd.read_csv(final_csv, index_col=0)
        df_curr.loc[len(df_curr.index)] = [url, text]
        # df_curr.concat(df_row, ignore_index=True)
        df_curr.to_csv(final_csv)  
    
# df_new = {"url" : list(df.url), 
#            "full_text": full_texts}
# df_text = pd.DataFrame(df_new)
# df_text.to_csv(f'NYT_text_data_final.csv')  


823 :  https://www.nytimes.com/2023/11/21/pageoneplus/corrections-nov-22-2023.html
824 :  https://www.nytimes.com/2023/11/13/pageoneplus/corrections-nov-14-2023.html
825 :  https://www.nytimes.com/video/world/europe/100000009176729/uk-london-palestinian-protest.html
826 :  https://www.nytimes.com/video/opinion/100000009160765/gaza-palestine-children-dreams.html
827 :  https://www.nytimes.com/2023/11/02/magazine/poem.html
828 :  https://www.nytimes.com/2023/11/03/opinion/gaza-palestine-children-dreams.html
829 :  https://www.nytimes.com/2023/11/01/arts/television/late-night-ron-desantis-height.html
830 :  https://www.nytimes.com/2023/11/30/arts/music/henry-kissinger-nixon-in-china-opera.html
831 :  https://www.nytimes.com/2023/11/07/nyregion/new-york-city-election.html
832 :  https://www.nytimes.com/2023/11/08/t-magazine/jorge-zalszupin-brazilian-modernism.html
833 :  https://www.nytimes.com/2023/11/09/us/hamas-hostages.html
834 :  https://www.nytimes.com/2023/11/19/pageoneplus/quotatio

missing article [240] from nov nyt scrape: 
https://www.nytimes.com/video/embedded/world/middleeast/100000009190139/indonesian-hospital-patients-transported-to-nasser-hospital.html